# Importing Essential Libraries

In [ ]:
import sys, os

import numpy as np
import pandas as pd
import geopandas as gpd
import shapely

In [ ]:
from st_visualizer import st_visualizer
import callbacks
import express as viz_express
import geom_helper as viz_helper

In [ ]:
import bokeh.models as bokeh_models

# Defining Global Variables

In [ ]:
NOTEBOOK_URL='http://<NOTEBOOK_IP_ADDRESS>:<NOTEBOOK_PORT>'

  * # Visualizing a Simple Geometry Dataset

In [ ]:
st_viz = st_visualizer(limit=500)
st_viz.get_data_csv(filepath='./data/csv/ais_brest_2015-2016.csv', nrows=30000)

st_viz.create_canvas(title=f'Prototype Plot', sizing_mode='scale_width', plot_height=540, tools="pan,box_zoom,lasso_select,wheel_zoom,previewsave,reset")

st_viz.add_map_tile('CARTODBPOSITRON')
circ = st_viz.add_glyph(glyph_type='circle', size=10, color='royalblue', alpha=0.7, fill_alpha=0.5, muted_alpha=0, legend_label=f'Vessel GPS Locations')

tooltips = [('Vessel ID','@mmsi'), ('Timestamp','@ts'), ('Speed (knots)','@speed'),('Course over Ground (degrees)','@course'), ('Heading (degrees)','@heading'), ('Coordinates','(@lon, @lat)')]
st_viz.add_hover_tooltips(tooltips)
st_viz.add_lasso_select()

st_viz.figure.legend.location = "top_left"
st_viz.figure.legend.click_policy = "mute"
st_viz.figure.toolbar.active_scroll = st_viz.figure.select_one(bokeh_models.WheelZoomTool)

st_viz.show_figures(notebook=True, notebook_url=NOTEBOOK_URL)

### (ADVANCED) Responsive Canvas

In [ ]:
st_viz = st_visualizer(limit=500)
st_viz.get_data_csv(filepath='./data/csv/ais_brest_2015-2016.csv', nrows=30000)

st_viz.create_canvas(title=f'Prototype Plot', sizing_mode='stretch_width', plot_height=540, tools="pan,box_zoom,lasso_select,wheel_zoom,previewsave,reset")

st_viz.add_map_tile('CARTODBPOSITRON')
circ = st_viz.add_glyph(glyph_type='circle', size=10, color='royalblue', alpha=0.7, fill_alpha=0.5, muted_alpha=0, legend_label=f'Vessel GPS Locations')

tooltips = [('Vessel ID','@mmsi'), ('Timestamp','@ts'), ('Speed (knots)','@speed'),('Course over Ground (degrees)','@course'), ('Heading (degrees)','@heading'), ('Coordinates','(@lon, @lat)')]
st_viz.add_hover_tooltips(tooltips)
st_viz.add_lasso_select()

st_viz.figure.legend.location = "top_left"
st_viz.figure.legend.click_policy = "mute"
st_viz.figure.toolbar.active_scroll = st_viz.figure.select_one(bokeh_models.WheelZoomTool)

st_viz.show_figures(notebook=True, notebook_url=NOTEBOOK_URL, sizing_mode='stretch_both')

  * # Adding Filters
    * #### Temporal Filter

In [ ]:
st_viz = st_visualizer(limit=500)
st_viz.get_data_csv(filepath='./data/csv/ais_brest_2015-2016.csv', nrows=30000)

tooltips = [('Vessel ID','@mmsi'), ('Timestamp','@ts'), ('Speed (knots)','@speed'),
            ('Course over Ground (degrees)','@course'), ('Heading (degrees)','@heading'), ('Coordinates','(@lon, @lat)')]

viz_express.plot_points_on_map(st_viz, tools=['lasso_select'], tooltips=tooltips)

st_viz.add_temporal_filter(temporal_name='ts', temporal_unit='s', step_ms=500, title='Temporal Horizon')

st_viz.show_figures(notebook=True, notebook_url=NOTEBOOK_URL)

  * #### Numerical Filter (range)

In [ ]:
st_viz = st_visualizer(limit=500)
st_viz.get_data_csv(filepath='./data/csv/ais_brest_2015-2016.csv', nrows=30000)

tooltips = [('Vessel ID','@mmsi'), ('Timestamp','@ts'), ('Speed (knots)','@speed'),
            ('Course over Ground (degrees)','@course'), ('Heading (degrees)','@heading'), ('Coordinates','(@lon, @lat)')]

viz_express.plot_points_on_map(st_viz, tools=['lasso_select'], tooltips=tooltips)

st_viz.add_numerical_filter(filter_mode='range', title='Speed (knots)', numeric_name='velocity', step=1)

st_viz.show_figures(notebook=True, notebook_url=NOTEBOOK_URL)

  * #### Numerical Filter (>=)

In [ ]:
st_viz = st_visualizer(limit=500)
st_viz.get_data_csv(filepath='./data/csv/ais_brest_2015-2016.csv', nrows=30000)

tooltips = [('Vessel ID','@mmsi'), ('Timestamp','@ts'), ('Speed (knots)','@speed'),
            ('Course over Ground (degrees)','@course'), ('Heading (degrees)','@heading'), ('Coordinates','(@lon, @lat)')]

viz_express.plot_points_on_map(st_viz, tools=['lasso_select'], tooltips=tooltips)

st_viz.add_numerical_filter(filter_mode='>=', title='Speed (knots) >=', numeric_name='velocity', step=1)

st_viz.show_figures(notebook=True, notebook_url=NOTEBOOK_URL)

  * #### Categorical

In [ ]:
glife = pd.read_csv('./data/csv/geolife_trips_cleaned_v2_china_subset.csv', nrows=50000)
glife = viz_helper.getGeoDataFrame_v2(glife, crs='epsg:4326')

st_viz = st_visualizer(limit=500)
st_viz.set_data(glife)

tooltips = [('User ID','@user_id'), ('Vehicle','@label')]

viz_express.plot_points_on_map(st_viz, tools=['lasso_select'], tooltips=tooltips)


st_viz.add_categorical_filter(title='Vehicle', categorical_name='label')

st_viz.show_figures(notebook=True, notebook_url=NOTEBOOK_URL)

# A Note Towards Filter Interaction

In [ ]:
world_airports = pd.read_csv('data/csv/world_airports.csv')
world_airports = viz_helper.getGeoDataFrame_v2(world_airports, coordinate_columns=['Longitude', 'Latitude'], crs='epsg:4326')

''' For webmercator, when latitudes tend to 90 deg, northing tend to infinity, so using 89.9999 is not the solution. 
    The recommended area of use of EPSG:3857 is for latitudes between -85 and 85 degrees.
    Thus, we get the locations that reside anywhere but the poles. '''

world_airports = world_airports.loc[world_airports.Latitude.between(-85, 85)].copy()
world_airports = world_airports.loc[world_airports.Altitude >= 0].copy()

In [ ]:
st_viz = st_visualizer(limit=500)
st_viz.set_data(world_airports, sp_columns=['Longitude', 'Latitude'])

tooltips = [('Name','@Name'), ('City Country','@City_Country'), ('IATA','@IATA'), ('Location','(@Longitude, @Latitude, @Altitude)'), ('Timezone', '@Timezone.1')]
viz_express.plot_points_on_map(st_viz, size=5, tools=['hover,lasso_select'], tooltips=tooltips)

st_viz.add_categorical_filter()
st_viz.add_numerical_filter(filter_mode='>=', callback_policy='value_throttled')

st_viz.figure.legend.location = "top_left"
st_viz.figure.legend.click_policy = "mute"
st_viz.figure.toolbar.active_scroll = st_viz.figure.select_one(bokeh_models.WheelZoomTool)

st_viz.show_figures(notebook=True, notebook_url=NOTEBOOK_URL)

# Α Note Towards (Multi)Polygon Visualization

In [ ]:
url = 'http://polygons.openstreetmap.fr/get_geojson.py?id=9135811&params=0'
saronikos = gpd.read_file(url)
saronikos.loc[:, saronikos.geometry.name] = saronikos.geometry.apply(lambda l: l[0])

geom = saronikos.geometry[0]
geom_final = geom[0]

geom_final = shapely.geometry.Polygon(geom_final.exterior.coords, [inner.exterior.coords for inner in geom[1:]])

saronikos_gulf = saronikos.copy()
saronikos_gulf.loc[:, saronikos_gulf.geometry.name] = geom_final

  * ## Visualizing Simple Polygons

In [ ]:
st_viz = st_visualizer(allow_complex_geometries=False)
st_viz.set_data(saronikos_gulf)

viz_express.plot_polygons_on_map(st_viz, polygon_type='patches')

st_viz.show_figures(notebook=True, notebook_url=NOTEBOOK_URL)

  * ## Visualizing Complex Geometries

In [ ]:
st_viz = st_visualizer(allow_complex_geometries=True)
st_viz.set_data(saronikos_gulf)

viz_express.plot_polygons_on_map(st_viz, polygon_type='multi_polygons')

st_viz.show_figures(notebook=True, notebook_url=NOTEBOOK_URL)